In [59]:
import numpy as np
from collections import OrderedDict
import logging

from IPython.display import display
%matplotlib inline

from astropy.io import fits
import astropy.wcs
from astropy import coordinates
import astropy.units as apu
from astropy import table

import astropyp

alogger = logging.getLogger('astropyp.astrometry')
alogger.setLevel(logging.INFO)

idx_connect = 'sqlite:////media/data-beta/users/fmooleka/decam/decam.db'

conv_filter = np.array([
    [0.030531, 0.065238, 0.112208, 0.155356, 0.173152, 0.155356, 0.112208, 0.065238, 0.030531],
    [0.065238, 0.139399, 0.239763, 0.331961, 0.369987, 0.331961, 0.239763, 0.139399, 0.065238],
    [0.112208, 0.239763, 0.412386, 0.570963, 0.636368, 0.570963, 0.412386, 0.239763, 0.112208],
    [0.155356, 0.331961, 0.570963, 0.790520, 0.881075, 0.790520, 0.570963, 0.331961, 0.155356],
    [0.173152, 0.369987, 0.636368, 0.881075, 0.982004, 0.881075, 0.636368, 0.369987, 0.173152],
    [0.155356, 0.331961, 0.570963, 0.790520, 0.881075, 0.790520, 0.570963, 0.331961, 0.155356],
    [0.112208, 0.239763, 0.412386, 0.570963, 0.636368, 0.570963, 0.412386, 0.239763, 0.112208],
    [0.065238, 0.139399, 0.239763, 0.331961, 0.369987, 0.331961, 0.239763, 0.139399, 0.065238],
    [0.030531, 0.065238, 0.112208, 0.155356, 0.173152, 0.155356, 0.112208, 0.065238, 0.030531]
])
# SExtractor 'extract' detection parameters
sex_params = {
    'extract': {
        #'thresh': 1.5,# *bkg.globalrms,
        #'err':,
        #'minarea': 5, # default
        'conv': conv_filter,
        #'deblend_nthresh': 32, #default
        'deblend_cont': 0.001,
        #'clean': True, #default
        #'clean_param': 1 #default
    },
    'kron_k': 2.5,
    'kron_min_radius': 3.5,
    'filter': conv_filter,
    'thresh': 1.5
}

from astropyp.wrappers.astromatic import ldac
obj='F100'
refname = '2MASS'
#refname = 'UCAC4'
fullref = ldac.get_table_from_ldac(
    '/media/data-beta/users/fmooleka/decam/catalogs/ref/{0}-{1}.fits'.format(obj, refname))

refcat = astropyp.catalog.Catalog(fullref)
#refcat = astropyp.catalog.Catalog(fullref, ra='ra_J2015.4', dec='dec_J2015.4', 
#                                  e_ra='e_ra_J2015.4',e_dec='e_dec_J2015.4')

In [39]:
import warnings
from astropy.utils.exceptions import AstropyWarning
warnings.simplefilter('ignore', category=AstropyWarning)

def get_exp_files(expnum, night, filtr, idx_connect):
    sql = 'select * from decam_obs where expnum={0}'.format(expnum)
    exp_info = astropyp.db_utils.index.query(sql, idx_connect)
    img_filename = exp_info[exp_info['PRODTYPE']=='image'][0]['filename']
    img = fits.open(img_filename)
    dqmask_filename = exp_info[exp_info['PRODTYPE']=='dqmask'][0]['filename']
    dqmask = fits.open(dqmask_filename)
    return img, dqmask
  
min_flux = 1000
min_amplitude = 1000
good_amplitude = 100
calibrate_amplitude = 200
frame = 1
#explist = [442430, 442431, 442432]
#explist = [442433, 442434, 442435]
#explist = [442428]
explist = [206401]

ccds = OrderedDict()
good_indices = OrderedDict()
allwcs = OrderedDict()
imgs = []
for expnum in explist:
    #img, dqmask = get_exp_files(expnum, "2015-05-26", "i", idx_connect)
    img, dqmask = get_exp_files(expnum, "2015-05-26", "z", idx_connect)
    imgs.append(img[frame].data)
    header = img[frame].header
    wcs = astropy.wcs.WCS(header)
    img_data = img[frame].data
    dqmask_data = dqmask[frame].data
    ccd = astropyp.phot.phot.SingleImage(header, img_data, dqmask_data,
        wcs=wcs, gain=4., exptime=30, aper_radius=8)
    ccd.detect_sources(sex_params, subtract_bkg=True)
    ccd.select_psf_sources(min_flux,min_amplitude)
    good_idx = ccd.catalog.sources['peak']>calibrate_amplitude
    good_idx = good_idx & (ccd.catalog.sources['pipeline_flags']==0)
    alogger.info("Good sources: {0}".format(np.sum(good_idx)))
    ccds[expnum] = ccd
    good_indices[expnum] = good_idx
    allwcs[expnum] = wcs

INFO:astropyp.phot.psf:Total sources: 1870
INFO:astropyp.phot.psf:Sources with low flux: 192
INFO:astropyp.phot.psf:Sources with low amplitude: 1624
INFO:astropyp.phot.psf:Sources with bad pixels: 295
INFO:astropyp.phot.psf:Elliptical sources: 161
INFO:astropyp.phot.psf:Source with close neighbors: 547
INFO:astropyp.phot.psf:Sources near an edge: 15
INFO:astropyp.phot.psf:Sources after cuts: 142
INFO:astropyp.astrometry:Good sources: 390


In [68]:
#catalogs = [ccd.catalog.sources[good_indices[expnum]] for expnum, ccd in ccds.items()]
#cat1, cat2, cat3 = astropyp.catalog.match_all_catalogs(catalogs, 'ra','dec', combine=False)
cat1 = ccds[206401].catalog.sources[good_indices[206401]]

good_cat = astropyp.catalog.Catalog(cat1, ra='ra_win', dec='dec_win', x='xwin',y='ywin')

crpix = [1024, 2048]
cra, cdec = wcs.wcs_pix2world([crpix[0]],[crpix[1]],0)
crval = [cra[0],cdec[0]]

metric = astropyp.astrometry.AstrometricSolution(crpix=crpix, crval=crval)

ref_weights = np.array(1/np.sqrt(refcat.e_ra**2+refcat.e_dec**2))
#weights = good_cat['aper_flux']/good_cat['aper_flux_err']

idx, matched = metric.get_solution(good_cat, refcat, ref_weights=ref_weights)
#idx, matched = metric.get_solution(good_cat, refcat, separation=3*apu.arcsec)

ra,dec = metric.pix2world(good_cat.x[matched],good_cat.y[matched])

d_ra = ((np.abs(ra-np.array(refcat.ra[idx][matched]))*np.cos(np.deg2rad(np.array(dec))))*apu.deg).to('mas')
d_dec = ((np.abs(dec-np.array(refcat.dec[idx][matched])))*apu.deg).to('mas')
print 'mean d_ra', np.mean(d_ra)
print 'mean d_dec', np.mean(d_dec)
print 'ra rms', np.sqrt(np.sum(d_ra**2)/d_ra.shape[0])
print 'dec rms', np.sqrt(np.sum(d_dec**2)/d_dec.shape[0])

c1 = coordinates.SkyCoord(ra, dec, unit='deg')
c2 = coordinates.SkyCoord(refcat.ra[idx][matched], refcat.dec[idx][matched], unit='deg')
separation = c1.separation(c2)
separation = separation.to('arcsec')
print np.mean(separation)
print np.std(separation)
print 'rms', np.sqrt(np.sum(separation**2)/separation.shape[0])

#pidx = good_cat['peak'][matched]>2000
pidx = good_cat['aper_flux'][matched]/good_cat['aper_flux_err'][matched]>100
c1 = coordinates.SkyCoord(ra[pidx], dec[pidx], unit='deg')
c2 = coordinates.SkyCoord(refcat.ra[idx][matched][pidx], refcat.dec[idx][matched][pidx], unit='deg')
separation = c1.separation(c2)
separation = separation.to('arcsec')
print 'high SNR sources', np.sum(pidx)
print 'high SNR', np.mean(separation)
print 'high SNR', np.std(separation)
print 'high SNR rms', np.sqrt(np.sum(separation**2)/separation.shape[0])

x,y = metric.world2pix(ra, dec)
d2 = np.sqrt((x-good_cat.x[matched])**2 + (y-good_cat.y[matched])**2)
print 'cartesian mean', np.mean(d2)
print 'cartesian stddev', np.std(d2)
print 'cartesian rms', np.sqrt(np.sum(d2**2)/d2.shape[0])

INFO:astropyp.astrometry:formula: "ra ~ A_1_0+A_0_1+A_2_0+A_1_1+A_0_2+A_3_0+A_2_1+A_1_2+A_0_3"
INFO:astropyp.astrometry:formula: "dec ~ B_1_0+B_0_1+B_2_0+B_1_1+B_0_2+B_3_0+B_2_1+B_1_2+B_0_3"
INFO:astropyp.astrometry:formula: "x ~ Ap_1_0+Ap_0_1+Ap_2_0+Ap_1_1+Ap_0_2+Ap_3_0+Ap_2_1+Ap_1_2+Ap_0_3"
INFO:astropyp.astrometry:formula: "y ~ Bp_1_0+Bp_0_1+Bp_2_0+Bp_1_1+Bp_0_2+Bp_3_0+Bp_2_1+Bp_1_2+Bp_0_3"


mean d_ra 152.561464456 mas
mean d_dec 162.184190601 mas
ra rms 212.447627043 mas
dec rms 231.882186944 mas
0.246299arcsec
0.195551arcsec
rms 0.314489059655 arcsec
high SNR sources 228
high SNR 0.246299arcsec
high SNR 0.195551arcsec
high SNR rms 0.314489059655 arcsec
cartesian mean 0.00201935257059
cartesian stddev 0.00120654493732
cartesian rms 0.00235234680481


In [69]:
d_ra = refcat.e_ra[idx][matched]
d_dec = refcat.e_dec[idx][matched]
print np.mean(d_ra), np.mean(d_dec)
print np.sqrt(np.sum(d_ra**2)/d_ra.shape[0])
print np.sqrt(np.sum(d_dec**2)/d_dec.shape[0])

168.289472823 mas 142.192982774 mas
185.620107265 mas
159.373774689 mas
